In [1]:
%pip install ultralytics
%pip install torch
%pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import torch
from ultralytics import YOLO
from time import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage  # Import for attaching images
from ultralytics.utils.plotting import Annotator, colors
from email_settings import from_email, password, to_email
import os

WARNING ⚠️ Python>=3.10 is required, but Python==3.9.6 is currently installed 


/Users/admin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
#setting up email server
server = smtplib.SMTP("smtp.gmail.com: 587")
server.starttls()
server.login(from_email, password)

(235, b'2.7.0 Accepted')

In [4]:
#directory to save image
save_dir = "detected_images"
os.makedirs(save_dir, exist_ok=True)


In [5]:
password = "kggz ayio yafp zfvz"
from_email = "joswinshogi@gmail.com"
to_email = "print4525@gmail.com" 

In [6]:
#function for sending email
def send_email(to_email, from_email, object_detected=1, image_path=None):
    """Sends an email notification indicating the number of objects detected; defaults to 1 object."""
    message = MIMEMultipart()
    message["From"] = from_email
    message["To"] = to_email
    message["Subject"] = "Security Alert"
    
    # Add in the message body
    message_body = f"ALERT - {object_detected} object{'s' if object_detected > 1 else ''} has been detected!!"
    message.attach(MIMEText(message_body, "plain"))

    # Attach the detected image if provided
    if image_path:
        with open(image_path, 'rb') as img_file:
            img = MIMEImage(img_file.read())
            img.add_header('Content-Disposition', f'attachment; filename="{os.path.basename(image_path)}"')
            message.attach(img)

    server.sendmail(from_email, to_email, message.as_string())

In [7]:
#saving detected image
def detected_image(frame):
    """Saves the detected image in the specified directory and returns its path."""
    timestamp = int(time())
    image_path = os.path.join(save_dir, f"detected_person_{timestamp}.jpg")
    cv2.imwrite(image_path, frame)
    print(f"Saved detected image: {image_path}")
    return image_path  # Return the path for emailing

In [8]:
class ObjectDetection:
    def __init__(self, capture_index):
        """Initializes an ObjectDetection instance with a given camera index."""
        self.capture_index = capture_index
        self.email_sent = False
        self.image_counter = 0  # Counter for sent images

        # Model information
        self.model = YOLO("yolo11n.pt")

        # Visual information
        self.annotator = None

        # Device information
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    # ... (rest of your existing ObjectDetection code)

    def predict(self, im0):
        """Run prediction using a YOLO model for the input image `im0`."""
        results = self.model(im0)
        return results

    def display_fps(self, im0):
        """Displays the FPS on an image `im0` by calculating and overlaying as white text on a black rectangle."""
        end_time = time()
        fps = 1 / round(end_time - self.start_time, 2)
        text = f"FPS: {int(fps)}"
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
        gap = 10
        cv2.rectangle(
            im0,
            (20 - gap, 70 - text_size[1] - gap),
            (20 + text_size[0] + gap, 70 + gap),
            (255, 255, 255),
            -1,
        )
        cv2.putText(im0, text, (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)

    def plot_bboxes(self, results, im0):
        """Plots bounding boxes on an image given detection results; returns annotated image and class IDs."""
        class_ids = []
        self.annotator = Annotator(im0, 3, results[0].names)
        boxes = results[0].boxes.xyxy.cpu()
        clss = results[0].boxes.cls.cpu().tolist()
        names = results[0].names
        
        for box, cls in zip(boxes, clss):
            class_ids.append(cls)
            self.annotator.box_label(box, label=names[int(cls)], color=colors(int(cls), True))
            
            # Save image if a person is detected (assuming class ID for person is 0)
            if cls == 0:  
                img_path = detected_image(im0)  # Save the current frame when a person is detected

                # Calculate center point of the bounding box for tracking
                x_center = int((box[0] + box[2]) / 2)
                y_center = int((box[1] + box[3]) / 2)

                # Draw a point on the center of the detected person
                cv2.circle(im0, (x_center, y_center), radius=5, color=(255, 0, 0), thickness=-1)  

            if self.image_counter < 5:
                
                send_email(to_email, from_email, len(class_ids), img_path)
                self.image_counter += 1  # Send email with attached image
                
        return im0, class_ids

    def __call__(self):
        """Run object detection on video frames from a camera stream, plotting and showing the results."""
        cap = cv2.VideoCapture(self.capture_index)
        
        assert cap.isOpened()
        
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        while True:
            self.start_time = time()
            ret, im0 = cap.read()
            assert ret
            
            results = self.predict(im0)
            im0, class_ids = self.plot_bboxes(results, im0)

            if len(class_ids) > 0:  
                if not self.email_sent:
                    send_email(to_email, from_email, len(class_ids))
                    self.email_sent = True
            else:
                self.email_sent = False

            self.display_fps(im0)
            cv2.imshow("YOLO11 Detection", im0)

            if cv2.waitKey(5) & 0xFF == 27:  
                break
        
        cap.release()
        cv2.destroyAllWindows()
        server.quit()

In [9]:
if __name__ == "__main__":
    detector = ObjectDetection(capture_index=1)
    detector()

2024-10-14 20:25:11.357 Python[81700:7698229] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.



0: 480x640 1 person, 98.4ms
Speed: 5.8ms preprocess, 98.4ms inference, 6.7ms postprocess per image at shape (1, 3, 480, 640)
Saved detected image: detected_images/detected_person_1728917713.jpg

0: 480x640 1 person, 1 chair, 1 cell phone, 82.4ms
Speed: 1.5ms preprocess, 82.4ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
Saved detected image: detected_images/detected_person_1728917717.jpg

0: 480x640 1 person, 1 chair, 58.5ms
Speed: 1.2ms preprocess, 58.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
Saved detected image: detected_images/detected_person_1728917724.jpg

0: 480x640 1 person, 1 chair, 61.9ms
Speed: 1.4ms preprocess, 61.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
Saved detected image: detected_images/detected_person_1728917726.jpg

0: 480x640 1 person, 1 chair, 48.7ms
Speed: 1.3ms preprocess, 48.7ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
Saved detected image: detected_images/detect

2024-10-14 20:25:27.060 Python[81700:7698229] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-14 20:25:27.060 Python[81700:7698229] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Saved detected image: detected_images/detected_person_1728917727.jpg

0: 480x640 1 person, 1 chair, 48.4ms
Speed: 1.1ms preprocess, 48.4ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
Saved detected image: detected_images/detected_person_1728917727.jpg

0: 480x640 1 person, 1 chair, 45.4ms
Speed: 1.2ms preprocess, 45.4ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
Saved detected image: detected_images/detected_person_1728917727.jpg

0: 480x640 1 person, 1 chair, 46.7ms
Speed: 1.1ms preprocess, 46.7ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
Saved detected image: detected_images/detected_person_1728917727.jpg

0: 480x640 1 person, 1 chair, 51.7ms
Speed: 1.2ms preprocess, 51.7ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)
Saved detected image: detected_images/detected_person_1728917727.jpg

0: 480x640 1 person, 1 chair, 47.8ms
Speed: 1.3ms preprocess, 47.8ms inference, 0.4ms postprocess per image at sha

KeyboardInterrupt: 

: 